In [20]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import Series
import os 
from itertools import product

import warnings

#from modshogun import *

from sklearn import linear_model, decomposition
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, GroupKFold, LeaveOneGroupOut
from sklearn.externals.joblib import Parallel, delayed
from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler, Imputer, MinMaxScaler
from sklearn.pipeline import Pipeline

from CustomCVs import KFoldMixedSizes, StratifiedKFoldMixedSizes, StratifiedKFoldByGroups
#from evaluation_classifier import Evaluater

from time import time
from IPython.display import clear_output

#from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, IterativeSVD #, MICE

from six.moves import cPickle as pickle

import matplotlib.pyplot as plt

In [2]:
# def create_rank_k_dataset(
#         n_rows=5,
#         n_cols=5,
#         k=3,
#         fraction_missing=0.1,
#         symmetric=False,
#         random_seed=0):
#     np.random.seed(random_seed)
#     x = np.random.randn(n_rows, k)
#     y = np.random.randn(k, n_cols)

#     XY = np.dot(x, y)

#     if symmetric:
#         assert n_rows == n_cols
#         XY = 0.5 * XY + 0.5 * XY.T

#     missing_raw_values = np.random.uniform(0, 1, (n_rows, n_cols))
#     missing_mask = missing_raw_values < fraction_missing

#     XY_incomplete = XY.copy()
#     # fill missing entries with NaN
#     XY_incomplete[missing_mask] = np.nan

#     return XY, XY_incomplete, missing_mask

# # create some default data to be shared across tests
# XY, XY_incomplete, missing_mask = create_rank_k_dataset(
#     n_rows=500,
#     n_cols=10,
#     k=3,
#     fraction_missing=0.25)

In [3]:
def create_correlated_dataset(cov_mat, n_obs = 2500):
    
    n_vars = cov_mat.shape[0]
    cov_mat = cov_mat + 1 * np.eye(n_vars) # regularize for stability
    
    try:
        L = np.linalg.cholesky(cov_mat)
        D = np.dot(L, np.random.uniform(0,1, (n_vars, n_obs)))
        return D
    
    except np.linalg.LinAlgError as err:
        print('Error ---- Cholesksy')
        return None
        
    

In [4]:
# test
cov_mat = np.array([[1, 0.7, 0.7, 0.5,],
             [0.7, 1, 0.95, 0.3],
             [0.7, 0.95, 1, 0.3],
             [0.5, 0.3, 0.3, 1]])

D = create_correlated_dataset(cov_mat)
print(D.shape)
np.corrcoef(D)

(4, 2500)


array([[1.        , 0.37224904, 0.36230115, 0.20353479],
       [0.37224904, 1.        , 0.48604921, 0.12585383],
       [0.36230115, 0.48604921, 1.        , 0.16007137],
       [0.20353479, 0.12585383, 0.16007137, 1.        ]])

In [5]:
# Construct a cov matrix using a REAL dataset

data_dir="/data/rmthomas/HeteroSmallSample"
df = pd.read_csv(os.path.join(data_dir, "real_data.csv"))
df_numeric = df[df.columns[25:125]]
cov_mat_overall = df_numeric.corr().values

alpha = 0.2
reg_cov_mat = cov_mat_overall + alpha*np.eye(100) # alpha makes the matrix well conditioned for Cholesky
D_overall = create_correlated_dataset(reg_cov_mat)
print(D_overall.shape)

(100, 2500)


In [6]:
n_features = 100 
g = df.groupby(['site', 'Dx', 'age_group'])[df.columns[25:25+n_features]]

Groups = list(g.indices.keys())

In [7]:
# Generate correlation matrices per set = (site, Dx, age_group)
corrs_per_set = g.corr().values.reshape(-1, n_features, n_features)
#corrs_per_set[np.where(np.isnan(corrs_per_set))] = 0.5

In [8]:
feature_labels = [f'f{i}' for i in range(100)] # f1, f2 ...f100
data_cols = ['site', 'Dx', 'Age_group'] + feature_labels

In [9]:
sim_data_all = pd.DataFrame(columns=data_cols) # initialize a dataframe
group_template = pd.DataFrame(columns=['site', 'Dx', 'Age_group']) # initialize a dataframe

In [72]:
sim_data=[]

min_subj = 5
max_subj = 70
for corr_i in range(corrs_per_set.shape[0]):
    n_obs=np.random.choice(np.arange(min_subj, max_subj))
    D = create_correlated_dataset(corrs_per_set[corr_i], n_obs=n_obs)

    if D is not None:
        sim_data_group = pd.DataFrame([list(Groups[corr_i])], columns=['site', 'Dx', 'Age_group'])
        sim_data_group = pd.concat([sim_data_group]*n_obs, ignore_index=True)
        
        sim_data_group_matrix = pd.DataFrame(D.T, columns=feature_labels)
        df_group = pd.concat([sim_data_group, sim_data_group_matrix], axis=1, ignore_index=False)
        sim_data_all = sim_data_all.append(df_group, ignore_index=True)
        sim_data.append(D)

Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy
Error ---- Cholesksy


In [76]:
sim_data_all['site'].unique()

array(['Arnold', 'Benedetti', 'Beucke', 'Brennan', 'Buitelaar', 'Cheng',
       'Fitzgerald', 'Gruner', 'Heuvel', 'Hirano', 'Hoexter', 'Huyser',
       'Koch', 'Kwon', 'KwonNMC', 'KwonSNU', 'Lazaro', 'Marsh',
       'Mataix_Cols', 'Menchon', 'Morgado', 'Nakamae', 'Nakao', 'Nurmi',
       'Reddy', 'Simpson', 'Soreni', 'Spalletta', 'Stein', 'Stewart',
       'Tolin', 'Walitza', 'Wang'], dtype=object)

In [78]:
sim_data_all.head()

,site,Dx,Age_group,f0,f1,f2,f3,f4,f5,f6,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,Arnold,1,2_pediatric,0.951022,1.512775,1.714066,2.085525,2.231103,1.757755,0.704306,...,-0.886497,0.890936,0.236274,0.526358,-0.286389,0.812563,-0.166830,0.611281,-0.342606,-0.269540
1,Arnold,1,2_pediatric,1.063553,0.551570,1.340835,1.222201,1.476826,0.992264,0.930643,...,-0.549631,0.704089,0.691332,-0.115608,0.188352,0.469429,0.342107,0.171521,0.303544,-1.086232
2,Arnold,1,2_pediatric,1.248328,0.760264,1.374774,0.790993,0.683645,0.987078,0.270414,...,-0.519217,0.439890,0.303501,0.510986,0.004420,0.359330,0.584917,0.718951,-0.409061,-0.553637
3,Arnold,1,2_pediatric,0.291278,0.569711,0.356906,0.871737,0.580628,0.767636,0.646040,...,-0.117505,0.902192,0.944374,0.074966,0.814591,-0.170715,0.317569,0.232148,0.518169,-0.931058
4,Arnold,1,2_pediatric,0.176449,0.717307,1.171825,1.235284,0.837378,1.454417,0.745621,...,-0.335837,0.598034,0.735613,0.051320,0.293834,0.840017,0.869743,0.593250,0.503287,-1.125220


In [81]:
print(sim_data_all[sim_data_all['Dx']==1].shape)
print(sim_data_all[sim_data_all['Dx']==0].shape)

(1213, 103)
(1337, 103)


In [ ]:
all_data = np.hstack(sim_data)
nvars, nsubjs = all_data.shape

In [ ]:
# Create data for each group
D_patients = create_correlated_dataset(cov_patients)
D_controls = create_correlated_dataset(cov_controls)

In [ ]:
import seaborn as sns
import matplotlib.gridspec as gs
import matplotlib.pyplot as plt
import itertools

a = 31
if a%2 != 0:
    a += 1

n = np.floor(np.sqrt(a)).astype(np.int64)

while a%n != 0:
    n -= 1

m = (a/n).astype(np.int64)
coords = list(itertools.product(list(range(m)), list(range(n))))




In [ ]:
from fancyimpute import MICE

In [ ]:
# https://www.kaggle.com/athi94/investigating-imputation-methods